In [ ]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser

def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [ ]:
headers = {
    'User-Agent': 'GOLUB CAPITAL BDC, Inc.'
}
filing_links = pd.read_excel(
    "../GBDC__sec_filing_links.xlsx")
filing_links.head(5)

In [ ]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head(5)

In [ ]:
# index=-2
# url, date = filing_links.iloc[index]['Filings URL'], filing_links.iloc[index]['Reporting date']
# print(url, date)
# response = requests.get(url, headers=headers)
# content = parse_and_trim(response.content, 'HTML')
# search_texts = [
#     'Consolidated Schedule of Investments',
#     'Consolidated Schedule of Investments - (continued)',
#         'Consolidated Schedule of Investments (unaudited) - (continued)',
#         'Consolidated Schedule of Investments (unaudited)']
# all_tags=content.find_all(text=search_texts)
# for tag in all_tags:
#     print(tag,tag.next)



In [ ]:
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(html_link, qtr_date)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    search_texts = [
    'Consolidated Schedule of Investments',
    'Consolidated Schedule of Investments - (continued)',
        'Consolidated Schedule of Investments (unaudited) - (continued)',
        'Consolidated Schedule of Investments (unaudited)',
        'Consolidated Schedule of Investments (unaudited) - continued',
        'Consolidated Schedule of Investments (unaudited) - (continued)',
        'Consolidated Schedule of Investments (unaudited)',
        'Consolidated Schedule of Investments (unaudited) - (continued)',
        'Consolidated Schedule of Investments - (continued)'
        ]
    pattern = r'Consolidated Schedule of Investments(?: \(-?continued\))? ?(?:- ?\(unaudited\))?'
    consolidated_schedule_regex = re.compile(r'^\s*.*\s*CONSOLIDATED\s+SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    # all_tags=content.find_all(text=consolidated_schedule_regex)
    # for tag in all_tags:
    for tag in content.find_all(text=pattern):
        print(tag,tag.next)
